In [1]:
%%capture
import os, importlib.util
!pip install --upgrade -qqq uv
if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):    
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    !uv pip install -qqq \
        "torch>=2.8.0" "triton>=3.4.0" {get_numpy} {get_pil} torchvision bitsandbytes "transformers==4.56.2" \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
        git+https://github.com/triton-lang/triton.git@0add68262ab0a2e33b84524346cb27cbb2787356#subdirectory=python/triton_kernels
elif importlib.util.find_spec("unsloth") is None:
    !uv pip install -qqq unsloth
!uv pip install --upgrade --no-deps torchinfo transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024
dtype = None

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="aitfindonesia/KomdigiUB-8B-Base",
    dtype=dtype,
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    full_finetuning=False,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.2: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

chat_template.jinja:   0%|          | 0.00/327 [00:00<?, ?B/s]

# Lora Adapter

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,    
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2026.1.2 patched 36 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [5]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 4096, padding_idx=151643)
        (layers): ModuleList(
          (0-35): 36 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
       

In [45]:
from torchinfo import summary

summary(model, depth=5)   # atur depth sesuai keinginan


Layer (type:depth-idx)                                            Param #
PeftModelForCausalLM                                              --
├─LoraModel: 1-1                                                  --
│    └─Qwen3ForCausalLM: 2-1                                      --
│    │    └─Qwen3Model: 3-1                                       --
│    │    │    └─Embedding: 4-1                                   (622,329,856)
│    │    │    └─ModuleList: 4-2                                  --
│    │    │    │    └─Qwen3DecoderLayer: 5-1                      97,689,856
│    │    │    │    └─Qwen3DecoderLayer: 5-2                      97,689,856
│    │    │    │    └─Qwen3DecoderLayer: 5-3                      97,689,856
│    │    │    │    └─Qwen3DecoderLayer: 5-4                      97,689,856
│    │    │    │    └─Qwen3DecoderLayer: 5-5                      97,689,856
│    │    │    │    └─Qwen3DecoderLayer: 5-6                      97,689,856
│    │    │    │    └─Qwen3DecoderLayer

## Non Thinking

In [24]:
model.generation_config.enable_thinking = False

In [46]:
from transformers import TextStreamer

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Apa itu Undang-Undang?"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True,
    reasoning_effort = "low",
    enable_thinking=False
).to("cuda")

output = model.generate(**inputs, max_new_tokens=1024, streamer=TextStreamer(tokenizer))

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Apa itu Undang-Undang?<|im_end|>
<|im_start|>assistant
<think>

</think>

Undang-Undang adalah peraturan perundang-undangan yang memiliki kekuatan hukum yang setara dengan peraturan perundang-undangan lainnya. Undang-undang merupakan peraturan perundang-undangan yang dibentuk oleh lembaga legislatif, biasanya dalam bentuk undang-undang, yang kemudian ditetapkan dan disahkan oleh lembaga negara yang berwenang, seperti Dewan Perwakilan Rakyat (DPR) di Indonesia. Undang-undang memiliki ciri-ciri sebagai berikut:<|im_end|>


## Thinking

In [48]:
model.generation_config.enable_thinking = True

In [49]:
from transformers import TextStreamer

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Apa itu Undang-Undang?"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True,
    reasoning_effort = "low",
).to("cuda")

output = model.generate(**inputs, max_new_tokens=1024, streamer=TextStreamer(tokenizer))

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Apa itu Undang-Undang?<|im_end|>
<|im_start|>assistant
<think>
Okay, so I need to explain what the Indonesian Constitution is. Let me start by recalling what I know. The Indonesian Constitution, or UUD, is the highest law in Indonesia. I think it was first established in 1945 when Indonesia gained independence. But wait, there have been amendments over time. The first amendment was in 1999, right? That's after the fall of Suharto's regime. Then there were more amendments in 2001 and 2002. The current version is the 1945 Constitution with these amendments. 

The main purpose of the Constitution is to outline the structure of the government, define the rights of citizens, and set the principles for the state. It establishes the three branches of government: executive, legislative, and judicial. The President is the head of state, and there's a Vice President. The legislative branch is the People's Representative C

# Data Preparation

In [57]:
def formatting_prompts(examples):
    questions = examples["question"]
    answers = examples["answer"]
    texts = []
    for question, answer in zip(questions, answers):
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": question},
            {"role": "assistant", "content": answer}
        ]
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        texts.append(text)
    return {"text": texts}

In [58]:
from datasets import load_dataset
from unsloth.chat_templates import standardize_sharegpt

dataset = load_dataset("AITF-Universitas-Brawijaya/prd2-data-sintetis", split="train")
dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 6347
})

In [59]:
dataset = dataset.map(formatting_prompts, batched=True)

Map:   0%|          | 0/6347 [00:00<?, ? examples/s]

In [67]:
# Check the actual formatted text to identify correct tokens
print("First formatted example:")
print(repr(dataset[0]["text"]))
print("\n" + "="*50 + "\n")
print("Readable version:")
print(dataset[0]["text"])

First formatted example:
"<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nSebagai seorang profesional database, saya perlu mengklasifikasikan data. Jelaskan definisi 'pop-up' pada website yang dicurigai sebagai situs judi online dari perspektif hukum.<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\nSecara teknis, 'pop-up' adalah jendela peramban yang muncul otomatis. Dalam konteks hukum, jika pop-up tersebut berisi ajakan, promosi, atau tautan ke permainan untung-untungan, maka ia berfungsi sebagai sarana 'membuat dapat diaksesnya' muatan perjudian, yang secara eksplisit dilarang oleh Pasal 27 ayat (2) UU No. 1 Tahun 2024 tentang Perubahan Kedua atas UU ITE.<|im_end|>\n"


Readable version:
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Sebagai seorang profesional database, saya perlu mengklasifikasikan data. Jelaskan definisi 'pop-up' pada website yang dicurigai sebagai situs judi online dari perspektif hukum.<|im_end|

# Train the Model

In [70]:
from trl import SFTConfig, SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",  # Specify the text field
    args=SFTConfig(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=16,
        warmup_ratio=0.03,
        num_train_epochs=1,
        learning_rate=2e-5,
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=42,
        output_dir="outputs",
        report_to="wandb"
    )
)

In [71]:
from unsloth.chat_templates import train_on_responses_only

# Correct tokens for ChatML format
trainer = train_on_responses_only(
    trainer,
    instruction_part="<|im_start|>user\n",
    response_part="<|im_start|>assistant\n",
)

Map (num_proc=6):   0%|          | 0/6347 [00:00<?, ? examples/s]

In [72]:
tokenizer.decode(trainer.train_dataset[100]['input_ids'])

"<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nBagaimana jika sebuah pop-up meminta izin notifikasi, dan setelah disetujui, notifikasi tersebut berisi promosi judi? Kapan pelanggaran hukumnya terjadi?<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\nPelanggaran hukum terjadi pada saat notifikasi berisi promosi judi itu 'didistribusikan' atau 'ditransmisikan' kepada pengguna. Ini adalah pelanggaran langsung terhadap Pasal 27 ayat (2) UU ITE.<|im_end|>\n"

In [73]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]['labels']]).replace(tokenizer.pad_token, " ")

"                                                                <think>\n\n</think>\n\nPelanggaran hukum terjadi pada saat notifikasi berisi promosi judi itu 'didistribusikan' atau 'ditransmisikan' kepada pengguna. Ini adalah pelanggaran langsung terhadap Pasal 27 ayat (2) UU ITE.<|im_end|>\n"

In [74]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.memory_reserved()/1024/1024/1024, 3)
max_memory = round(gpu_stats.total_memory/1024/1024/1024, 3)
print(f"GPU = {gpu_stats.name}, Max memory = {max_memory} GB")
print(f"{start_gpu_memory} GB of memory reserved")

GPU = Tesla T4, Max memory = 14.741 GB
6.369 GB of memory reserved


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6,347 | Num Epochs = 1 | Total steps = 397
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 16 x 1) = 16
 "-____-"     Trainable parameters = 43,646,976 of 8,234,382,336 (0.53% trained)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: Paste an API key from your profile and hit enter:wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: taqiyu

wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss


In [ ]:
used_memory = round(torch.cuda.max_memory_reserved()/1024/1024/1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = 